### Ch9_Security_Databrick's Free Dolly with LangChain

#### System Requirement:
- The transformers library on a machine with GPUs

#### Install Required Dependencies:
- accelerate>=0.16.0
- transformers[torch]>=4.28.1
- torch>=1.13.1
- lanchain

In [ ]:
!pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2" --quiet
#! install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"
!pip install langchain>=0.0.139 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 79.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have 

#### Import Dependencies and Use pipeline to load a custom InstructionTextGenerationPipeline

The instruction following pipeline can be loaded using the `pipeline` function as shown below. This loads a custom `InstructionTextGenerationPipeline`.

`torch_dtype=torch.bfloat16` is generally recommended to be used if this type is supported in order to reduce memory usage.

Let's use `dolly-v2-12b` model.

In [ ]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b",
                         torch_dtype=torch.bfloat16,
                         trust_remote_code=True,
                         device_map="auto",
                         return_full_text=True)

ImportError: ignored

You can then use the pipeline to answer instructions:

In [ ]:
res = generate_text("What is difference between New York and New York state?")
print(res[0]["generated_text"])

NameError: ignored

#### Import lanchain library functions to predict response based on provided Context

To use the pipeline with LangChain, you must set return_full_text=True, as LangChain expects the full text to be returned and the default for the pipeline is to only return the new text.

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

Template for an instrution with no input

In [ ]:
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

Template for an instruction with input

In [ ]:
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

In [ ]:
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)

In [ ]:
print(llm_chain.predict(instruction="Explain to me the difference between nuclear fission and fusion.").lstrip())

Nuclear fission occurs when a single atom splits in two, releasing large amounts of energy. The process takes free neutrons, which are released by the atom during this process.

Nuclear fusion, also known as plasma fusion, occurs when two or more atoms join together to form one or more different atomic nuclei. These nuclei may have the same mass or they may have different masses, and the mass difference is typically extremely small. In a fusion reaction, two or more neutrons are knocked loose, thereby increasing the mass of one of the resulting nuclei. This increased mass provides additional energy, which is usually released as light or beta particles.

The following are some of the differences between nuclear fusion and fission.
1. Fission releases free neutrons that can be used to generate heat or particles such as gamma rays, which cause the release of further fission neutrons.
2. Fusion converts chemical energy into nuclear energy, releasing light, heat, and particles, which could 

In [ ]:
context = """Lincoln's political career began in 1833 when he was elected to the Illinois House of Representatives.
Four years later, he was elected to the state Senate.  In 1847 he was elected Attorney General of Illinois, a position he held for ten years.
In 1859 he was elected to the United States Senate and served until 1861, when he was appointed to the US Supreme Court.
He served as Chief Justice from 1865-1868.
He returned to the US Senate in 1872 and served until 1875, when he was elected to his first Presidental office, serving as President from 1849-1850.
He became president once again in 1865-1869 and again in 1877-1881, this time of the University of Chicago.
He did not seek reelection in 1885.
During his time as president of the United States, he enacted the Civil War Amendments to the US Constitution, including the Thirteenth, Fourteenth and Fifteenth Amendments.
He also founded several historical landmarks including the University of Chicago and the World's Columbian Exposition in Chicago."""

print(llm_context_chain.predict(instruction="Summarize Lincoln's career?", context=context).lstrip())

Lincoln was a successful lawyer, state senator, state representative, Illinois Attorney General, US Senator and US Supreme Court Justice.  After losing his first presidential election, he was elected to the presidency three more times.  Lincoln was the 14th president of the US.


In [ ]:
context="""There are times when the night sky glows with bands of color. The bands may
begin as cloud shapes and then spread into a great arc across the entire sky. They
may fall in folds like a curtain drawn across the heavens. The lights usually grow
brighter, then suddenly dim. During this time the sky glows with pale yellow, pink,
green, violet, blue, and red. These lights are called the Aurora Borealis. Some
people call them the Northern Lights. Scientists have been watching them for
hundreds of years. They are not quite sure what causes them. In ancient times
people were afraid of the Lights. They imagined that they saw fiery dragons in the
sky. Some even concluded that the heavens were on fire."""
print(llm_context_chain.predict(instruction="Summarize the night sky related to {context}", context=context).lstrip())

The night sky glows with bands of color in order to best demonstrate. During this
time the sky glows with pale yellow, pink, green, violet, blue, and red.
